In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from functools import reduce
import numpy as np
from scipy.optimize import minimize

In [2]:
n = 2   # 两个设施点
m = 2   # 两个需求点
num_qubits = n + 2 * n * m

# # dij需求i到设施j的成本
# d = [[1, 2], [1, 2]]
# # gi设施i的建设成本
# g = [2, 1]

# dij需求i到设施j的成本
d = [[2, 1], [2, 1]]
# gi设施i的建设成本
g = [1, 2]

In [3]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def calculate_hamiltonian(v, w):
    n = len(v[0])
    m = len(v)
    hamiltonian = np.zeros((2**n, 2**n))
    for i in range(m):
        term1 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, v[i][j]) @ np.linalg.matrix_power(sigma_minus, w[i][j]) for j in range(n)])
        term2 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, w[i][j]) @ np.linalg.matrix_power(sigma_minus, v[i][j]) for j in range(n)])
        hamiltonian += term1 + term2
    return hamiltonian


def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2
    return Hp
        
Hp = generate_Hp(n, m, d, g)


In [4]:
import sys
sys.path.append('../../')
import z_library.linear_system as ls

In [5]:
def gnrt_gate_hdi(u):
    result = []
    nonzero_indices = ls.find_nonzero_indices(u)
    for urow, nrow in zip(u, nonzero_indices):
        # 把非0元素映射成01
        filtered_arr = [0 if x == -1 else 1 for x in urow if x != 0]
        binary_str = ''.join(map(str, filtered_arr))
        # 取到二进制表示的数
        map_num = int(binary_str, 2)
        # print(map_num)
        length = len(nrow)
        scale = 2**length
        matrix = np.zeros((scale, scale))

        matrix[map_num, scale - 1 - map_num] = 1
        map_num = 2**length - 1 - map_num
        # print(map_num)
        matrix[map_num, scale - 1 - map_num] = 1
        result.append(matrix)
    return result

# print("非零元素的索引：", nonzero_indices)

u = ls.gnrt_u(n, m)
print(u)
nonzero_indices = ls.find_nonzero_indices(u)
gate_hds = gnrt_gate_hdi(u)
print(nonzero_indices)

[[-1.  1. -1.  1. -1.  1.  0.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0. -1.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  1.  0.  1.  0.]
 [ 0.  1. -1.  1.  0.  0.  1.  0.  0.  1.]]
[[0, 1, 2, 3, 4, 5], [2, 3, 6, 7], [0, 6, 8], [1, 2, 3, 6, 9]]


In [6]:
gnrt_gate_hdi(u)[1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [7]:
from scipy.linalg import expm
def build_circ(params):
  depth = len(params) // 2
  qc = QuantumCircuit(num_qubits, num_qubits)
  beta = params[:depth]
  gamma = params[depth:]
  # for i in [1,3,4,6,8,9]:
  for i in [0, 1, 3, 5, 6, 8]:
    qc.x(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * beta[dp] * Hp), range(num_qubits)[::-1])
    for gate_hdi, ind in zip(gate_hds, nonzero_indices):
      qc.unitary(expm(-1j * gamma[dp] * gate_hdi), ind[::-1])
  qc.barrier()
  qc.measure(range(num_qubits), range(num_qubits)[::-1])
  return qc

In [8]:
ls.set_print_form()
build_circ(np.full(2 * 2, np.pi/3)).draw()

┌───┐┌──────────┐┌──────────┐            ┌──────────┐            »
 q_0: ┤ X ├┤9         ├┤5         ├────────────┤2         ├────────────»
      ├───┤│          ││          │            │          │┌──────────┐»
 q_1: ┤ X ├┤8         ├┤4         ├────────────┤          ├┤4         ├»
      └───┘│          ││          │┌──────────┐│          ││          │»
 q_2: ─────┤7         ├┤3         ├┤3         ├┤          ├┤3         ├»
      ┌───┐│          ││  Unitary ││          ││          ││          │»
 q_3: ┤ X ├┤6         ├┤2         ├┤2         ├┤          ├┤2         ├»
      └───┘│          ││          ││          ││          ││          │»
 q_4: ─────┤5         ├┤1         ├┤          ├┤  Unitary ├┤          ├»
      ┌───┐│  Unitary ││          ││  Unitary ││          ││          │»
 q_5: ┤ X ├┤4         ├┤0         ├┤          ├┤          ├┤  Unitary ├»
      ├───┤│          │└──────────┘│          ││          ││          │»
 q_6: ┤ X ├┤3         ├────────────┤1         ├┤1         ├┤1         ├»
      └───┘│          │            │          ││          ││          │»
 q_7: ─────┤2         ├────────────┤0         ├┤          ├┤          ├»
      ┌───┐│          │            └──────────┘│          ││          │»
 q_8: ┤ X ├┤1         ├────────────────────────┤0         ├┤          ├»
      └───┘│          │                        └──────────┘│          │»
 q_9: ─────┤0         ├────────────────────────────────────┤0         ├»
           └──────────┘                                    └──────────┘»
c: 10/═════════════════════════════════════════════════════════════════»
                                                                       »
«      ┌──────────┐┌──────────┐            ┌──────────┐             ░ ┌─┐      »
« q_0: ┤9         ├┤5         ├────────────┤2         ├─────────────░─┤M├──────»
«      │          ││          │            │          │┌──────────┐ ░ └╥┘┌─┐   »
« q_1: ┤8         ├┤4         ├────────────┤          ├┤4         ├─░──╫─┤M├───»
«      │          ││          │┌──────────┐│          ││          │ ░  ║ └╥┘┌─┐»
« q_2: ┤7         ├┤3         ├┤3         ├┤          ├┤3         ├─░──╫──╫─┤M├»
«      │          ││  Unitary ││          ││          ││          │ ░  ║  ║ └╥┘»
« q_3: ┤6         ├┤2         ├┤2         ├┤          ├┤2         ├─░──╫──╫──╫─»
«      │          ││          ││          ││          ││          │ ░  ║  ║  ║ »
« q_4: ┤5         ├┤1         ├┤          ├┤  Unitary ├┤          ├─░──╫──╫──╫─»
«      │  Unitary ││          ││  Unitary ││          ││          │ ░  ║  ║  ║ »
« q_5: ┤4         ├┤0         ├┤          ├┤          ├┤  Unitary ├─░──╫──╫──╫─»
«      │          │└──────────┘│          ││          ││          │ ░  ║  ║  ║ »
« q_6: ┤3         ├────────────┤1         ├┤1         ├┤1         ├─░──╫──╫──╫─»
«      │          │            │          ││          ││          │ ░  ║  ║  ║ »
« q_7: ┤2         ├────────────┤0         ├┤          ├┤          ├─░──╫──╫──╫─»
«      │          │            └──────────┘│          ││          │ ░  ║  ║  ║ »
« q_8: ┤1         ├────────────────────────┤0         ├┤          ├─░──╫──╫──╫─»
«      │          │                        └──────────┘│          │ ░  ║  ║  ║ »
« q_9: ┤0         ├────────────────────────────────────┤0         ├─░──╫──╫──╫─»
«      └──────────┘                                    └──────────┘ ░  ║  ║  ║ »
«c: 10/════════════════════════════════════════════════════════════════╩══╩══╩═»
«                                                                      9  8  7 »
«                           
« q_0: ─────────────────────
«                           
« q_1: ─────────────────────
«                           
« q_2: ─────────────────────
«      ┌─┐                  
« q_3: ┤M├──────────────────
«      └╥┘┌─┐               
« q_4: ─╫─┤M├───────────────
«       ║ └╥┘┌─┐            
« q_5: ─╫──╫─┤M├────────────
«       ║  ║ └╥┘┌─┐         
« q_6: ─╫──╫──╫─┤M├─────────
«       ║  ║  ║ └╥┘┌─┐      
« q_7: ─╫──╫──╫──╫─┤M├──────


In [9]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [10]:
# 初始化迭代计数器
iteration_count = 0
def test(par):
  global iteration_count
  iteration_count = 0
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, par, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.2f}%".format(counts[x] / shots * 100), cost_function(x))

In [11]:
for dep in range(1,10):
  print(f'depth == {dep}')
  test(np.full(dep * 2, np.pi/3))
  print()

depth == 1
Iteration 10, Result: 5.0015
Iteration 20, Result: 4.9985
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 4.9985
       x: [ 1.047e+00  4.720e-02]
    nfev: 23
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
1101011010 99.52% 5
0101010000 0.24% 4
1110010110 0.23% 6

depth == 2
Iteration 10, Result: 4.489
Iteration 20, Result: 4.003
Iteration 30, Result: 4.001
Iteration 40, Result: 4.0
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 4.0
       x: [ 1.028e+00  2.204e+00  1.573e+00  1.569e+00]
    nfev: 47
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
0101010000 100.00% 4
1101101001 0.00% 6
1110010110 0.00% 6

depth == 3
Iteration 10, Result: 4.758
Iteration 20, Result: 4.4445
It